## Loading the dataset

In [43]:
#!pip install sentence_transformers

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import numpy as np
import ast

In [7]:
# Set the maximum column width to 100 characters
pd.set_option('display.max_colwidth', 500)

In [8]:
books_df = pd.read_csv('books.csv', index_col=[0])

In [9]:
books_df.head(10)

,book_name,summaries,categories
0,The Highly Sensitive Person,"is a self-assessment guide and how-to-live template for people who feel, relate, process, and notice more deeply than others, and who frequently suffer from overstimulation as a result.",science
1,Why Has Nobody Told Me This Before?,"is a collection of a clinical psychologist’s best practical advice to combat anxiety and depression and improve our mental health in small increments, collected from over a decade of 1-on-1 work with patients.",science
2,The Midnight Library,"tells the story of Nora, a depressed woman in her 30s, who, on the day she decides to die, finds herself in a library full of lives she could have lived, where she discovers there’s a lot more to life, even her current one, than she had ever imagined.",science
3,Brave New World,"presents a futuristic society engineered perfectly around capitalism and scientific efficiency, in which everyone is happy, conform, and content — but only at first glance.",science
4,1984,is the story of a man questioning the system that keeps his futuristic but dystopian society afloat and the chaos that quickly ensues once he gives in to his natural curiosity and desire to be free.,science
5,Stolen Focus,"explains why our attention spans have been dwindling for decades, how technology accelerates this worrying trend, and what we can do to reclaim our focus and thus our capacity to live meaningful lives.",science
6,The Life-Changing Science of Detecting Bullshit,teaches its readers how to avoid falling for the lies and false information that other people spread by helping them build essential thinking skills through examples from the real world.,science
7,Dopamine Nation,"talks about the importance of living a balanced life in relation to all the pleasure and stimuli we’re surrounded with on a daily basis, such as drugs, devices, porn, gambling facilities, showing us how to avoid becoming dopamine addicts by restricting our access to them.",science
8,The Art of Statistics,"is a non-technical book that shows how statistics is helping humans everywhere get a new hold of data, interpret numbers, fact-check information, and reveal valuable insights, all while keeping the world as we know it afloat.",science
9,No Self No Problem,"is a provocative read about the implications of Buddhism in neuroscience, and more specifically about the idea that the self is only a product of the mind, meaning that there is no “I”.",science


In [10]:
books_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1227 entries, 0 to 1226
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   book_name   1227 non-null   object
 1   summaries   1227 non-null   object
 2   categories  1227 non-null   object
dtypes: object(3)
memory usage: 38.3+ KB


In [11]:
books_df.shape

(1227, 3)

## Embedding the summaries

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [13]:
# Step 1: Generate embeddings using BAAI/bge-small-en model from HuggingFace
model = SentenceTransformer("all-MiniLM-L6-v2")

In [14]:
# Generate embeddings for each summary
books_df['embedding'] = books_df['summaries'].map(lambda sentence: model.encode(sentence))

In [15]:
books_df.sample(10)

,book_name,summaries,categories,embedding
660,Unplug,"is your guide to utilizing meditation to enhance your brain, deal with stress, and become happier, explaining the basics of this practice, how to get started with it, and what science has to teach about its many benefits.",happiness,"[0.045592576, -0.006102961, 0.042459346, 0.015723646, -0.05685259, 0.088092834, 0.064080045, -0.055719696, -0.03855102, -0.01445886, -0.015351699, 0.05864657, -0.04132909, -0.050214186, 0.059103366, 0.004195667, -0.028084997, 0.061417338, -0.037611496, -0.011609139, -0.07830375, -0.010750204, -0.022242742, 0.021417653, -0.07138141, 0.07080512, -0.02782042, -0.060199745, 0.040889196, 0.024392184, 0.0685656, 0.04793823, 0.03513057, -0.03239323, -0.034567352, -0.004214814, -0.047470704, -0.0540..."
1127,Start Something That Matters,"encourages you to overcome your fear of the unknown and create a business that not only makes money but also helps people, even if you have few resources to start with.",marketing,"[0.004412756, 0.04561613, -0.008367388, 0.036618806, 0.020692535, 0.07045576, 0.09428892, -0.002848188, 0.036762506, -0.08128893, 0.025494296, 0.005433718, 0.06284812, -0.07242307, 0.0006406016, -0.037205014, 0.009506781, 0.040071923, -0.038263913, -0.09682972, -0.084899105, -0.03593197, -0.01705826, -0.0030635686, -0.05202382, 0.02631873, -0.01585871, -0.058735687, 0.060599003, -0.017864656, 0.0553466, 0.030707775, 0.08054442, -0.009345563, 0.011878823, 0.01260402, 0.0488614, 0.030924372, 0..."
411,Hug Your Haters,"talks about the importance of acknowledging your haters or dissatisfied customers and valuing their opinion in the process of building better products, improving the existing offerings, and growing your strategies overall.",relationships,"[-0.065660454, 0.04360758, 0.027753308, -0.04376792, -0.014279779, -0.0048213764, 0.10368902, 0.04601998, 0.027906476, -0.12608238, 0.011056388, 0.0119043095, 0.0709393, -0.04274144, 0.033766087, 0.03598525, 0.07139373, 0.024850592, -0.08121886, -0.076473504, -0.08255845, 0.0034661687, -0.016774176, 0.06857134, -0.09189179, -0.023833875, -0.039043684, 0.09465646, 0.00014102577, -0.07237552, -0.0035596364, 0.011282726, 0.12425484, 0.0026701307, -0.054792095, 0.028085908, -0.011399824, -0.0468..."
701,Designing Your Life,"will show you how to break the shackles of your mundane 9-5 job by sharing exercises and tips that will direct you towards your true calling that fills you with passion, purpose, and fulfillment.",happiness,"[-0.079854354, -0.04209636, 0.10032831, -0.033854708, 0.0017557701, 0.0055303, 0.0472083, -0.0040028836, -0.085442, -0.079234414, -0.08585105, 0.0845583, -0.064732864, 0.035297364, 0.02818544, -0.045832157, -0.04501339, 0.03961372, -0.06582608, -0.08066597, 0.040824756, 0.003454849, 0.011302328, 0.0069984943, 0.030043157, 0.0024658167, -0.010971408, -0.045969546, -0.0073552844, -0.06744926, 0.008030976, 0.07818355, -0.036576208, 0.07737664, -0.0014706507, 0.0856565, -0.041866094, 0.048376717..."
700,The Confidence Code,empowers women to become more courageous by explaining their natural tendencies toward timidity and how to break them even in a world dominated by men.,happiness,"[0.045022592, 0.038579617, 0.073602475, 0.10518812, 0.030860595, -0.0076989294, 0.010747663, -0.08880051, -0.0335676, 0.043366212, 0.0015630161, -0.036910217, -0.008033452, -0.08174252, 0.039190125, 0.03934225, 0.0018772357, 0.052590486, -0.086711936, 0.05984232, 0.028300164, -0.012037781, 0.047768336, 0.01593441, -0.08732879, 0.0064173294, -0.06952208, -0.006116282, 0.030167775, 0.010221099, -0.066432185, -0.006610486, 0.06854732, -0.026457584, -0.050075036, 0.07161713, -0.00038489987, 0.05..."
239,The Man Who Solved The Market,shares the interesting story of Jim Simons’s rise to wealth and success that came from him tapping into his math genius to make incredible gains in stock market investments.,biography,"[-0.0057029533, -0.0030460758, -0.039737307, -0.019290496, -0.060524784, -0.0015535959, 0.09

## Finding topN similar books

In [16]:
# Function to find top 5 similar books based on a given title
def get_cosine_simiarity(title, df):
    book_idx = books_df[books_df['book_name'] == title].index[0]
    book_embedding = books_df.loc[book_idx, 'embedding']
    similarities = cosine_similarity([book_embedding], books_df['embedding'].tolist()).flatten()
    return  similarities

### Finding Similar Books

 - The Bitcoin Standard
 - Measure What Matters
 - The Happiness Hypothesis

In [17]:
books_df.sample(10)['book_name']

372                            Moneyland
361       A Random Walk Down Wall Street
794                          Vagabonding
688     The 15 Invaluable Laws Of Growth
747                 The Presence Process
18                          Keto Answers
741                       The Happy Mind
845               Accounting Made Simple
233                             Educated
1153                 Blue Ocean Strategy
Name: book_name, dtype: object

In [18]:
# Display the titles of the top 5 similar movies for a given movie title
#sample_name = 'Einstein: His Life And Universe'
sample_name = "The Art Of Learning"
similarity = get_cosine_simiarity(sample_name,
                                  books_df)

In [19]:
rec_books = books_df.copy()

In [20]:
rec_books['similarity'] = list(similarity)

In [21]:
rec_books.sort_values('similarity', ascending = False)[0:6]

,book_name,summaries,categories,embedding,similarity
1015,The Art Of Learning,"explains the science of becoming a top performer, based on Josh Waitzkin’s personal rise to the top of the chess and Tai Chi world, by showing you the right mindset, proper ways to practice and how to build the habits of a professional.",productivity,"[0.05784928, -0.026134372, -0.018246029, -0.049593292, -0.027506119, 0.0735955, 0.121127754, 0.071278475, 0.010800556, 0.03754253, -0.0740603, -0.029962273, 0.046538424, 0.005211708, -0.008973105, 0.015000693, 0.04359592, 0.06203113, -0.011800744, -0.12551671, -0.07901497, -0.017937629, 0.10298973, -0.03364597, -0.1005688, -0.012524443, -0.047451388, 0.018569356, 0.06537605, -0.0032507589, 0.014864839, -0.023987722, 0.021213233, -0.007592734, -0.06327589, 0.08779866, -0.034875777, 0.04223542...",1.000000
192,Bounce,"shows you that training trumps talent every time, by explaining the science of deliberate practice, the mindset of high performers and how you can use those tools to become a master of whichever skill you choose.",science,"[0.078161195, 0.006413476, 0.07266931, -0.040888168, -0.017057516, 0.029580489, 0.07923102, -0.0002229142, -0.022682343, -0.060730115, -0.10109996, 0.01384271, 0.018788164, -0.028290654, -0.039159656, 0.005222504, -0.01140399, 0.017896678, -0.02605379, -0.07188087, 0.02859974, -0.033501055, 0.041888364, 0.007982245, -0.13575932, 0.043928374, -0.04126122, -0.028151171, 0.03917652, -0.036963653, 0.032730818, 0.0379503, -0.022033215, 0.02088661, -0.06034322, 0.042833, -0.040509444, 0.067659505,...",0.627209
1064,Psyched Up,is an in-depth look at the science behind mental preparation that will show you how to do your best when it counts the most based on what top performers do.,psychology,"[0.09361928, -0.06672085, -0.004758108, -0.04332279, -0.063986726, 0.06971201, 0.017052205, 0.013314338, 0.013974392, 0.025233412, -0.089240305, 0.024607243, -0.023643332, -0.02945854, 0.017636996, 0.010701143, 0.06665562, 0.00073538185, 0.00039695838, -0.042512953, 0.012702294, -0.027440391, 0.035632715, -0.019959558, -0.041987084, 0.0919612, -0.09838532, -0.067704506, 0.049813725, 0.013659849, -0.02648894, -0.014011197, 0.058583155, -0.021636525, -0.04443046, 0.10333964, -0.043961205, 0.06...",0.593900
985,Mind Gym,"explains why the performance of world-class athletes isn’t only a result of their physical training, but just as much due to their mentally fit minds and shows you how you can cultivate the mindset of a top performer yourself.",productivity,"[0.089482896, 0.052655768, 0.01833052, -0.07779843, 0.027507778, 0.015600005, 0.07273422, 0.036996264, 0.020507356, 0.039270658, -0.054540828, 0.0055625583, 0.009645466, -0.029610977, 0.033913884, -0.055170078, -0.005784613, 0.009797358, -0.07070636, 0.0013454193, -0.019187931, -0.01745367, 0.052600678, 0.04260827, -0.10801593, 0.059066847, -0.05147572, -0.012579833, -0.052748516, 0.030603452, -0.057812165, -0.036163256, -0.013445143, 0.01231579, -0.06644804, 0.07497149, -0.065787636, 0.0998...",0.581116
994,The Rise Of Superman,"decodes the science of ultimate, human performance by examining how top athletes enter and stay in a state of flow, while achieving their greatest feats, and how you can do the same.",productivity,"[0.01793601, 0.04562367, 0.0075618085, -0.056011267, 0.045094885, 0.04099075, 0.011155959, -0.0073525948, 0.015764412, 0.03763132, -0.12639749, -0.009437519, -0.04064674, 0.046559375, -0.031706166, -0.038416073, 0.003897604, 0.106662735, -0.042693216, -0.05529418, 0.06701721, -0.09042573, -0.010480888, 0.055190448, -0.07316479, 0.07505965, -0.10385108, 0.011831458, -0.028691778, -0.018821187, -0.013524418, -0.037276555, 0.071203984, 0.051780283, -0.14050335, 0.06547323, -0.033143263, 0.04409...",0.576935
956,Peak Performance,"shows you how to perform at your highest level by exploring the most significant factors that contribute to delivering our best work, such as stress, rest, focus, and purpose.",productivity,"[0.038

In [27]:
def get_similar_books(name, top_n = 5):
    similarity = get_cosine_simiarity(name,
                                      books_df)
    rec_books['similarity'] = list(similarity)

    return rec_books.sort_values('similarity', ascending = False)[1:top_n+1]

In [28]:
get_similar_books( 'Get A Financial Life' )

,book_name,summaries,categories,embedding,similarity
866,The Millionaire Next Door,shows you the simple spending and saving habits that lead to more cash in the bank than most people earn in their life while helping you avoid critical mistakes on your way to financial independence.,money,"[-0.009537507, 0.03984536, -0.04643181, 0.048608616, 0.04401241, -0.003606184, 0.10996673, 0.0366731, 0.017575836, -0.011498096, 0.012274362, -0.003930001, -0.015324695, -0.037045605, 0.03042797, -0.088015564, 0.0031616674, -0.063395485, -0.041718744, 0.009135882, -0.0026692087, -0.11201348, 0.012309487, 0.07683061, 0.0047930516, 0.015983947, -0.002792534, 0.0069618546, -0.0025294877, 0.0005399601, 0.07667864, 0.0639334, 0.06954505, -0.07818692, 0.0008735824, -0.001944305, 0.044840235, 0.107...",0.661513
832,Broke Millennial,shows those in their twenties and thirties how to manage their finances so that they can stop scraping by and instead begin to live more confidently when it comes to money.,money,"[0.016369442, 0.097227134, -0.0007255069, 0.05068167, 0.019502603, -0.0018904326, 0.007268181, 0.006958177, -0.037700776, 0.031602968, 0.06708081, -0.015058967, 0.02565856, -0.052371766, 0.01673205, -0.043298263, -0.048112787, -0.010302657, 0.05300575, -0.03328397, -0.036810435, -0.10629527, 0.05974952, -0.00037205397, 0.03049076, 0.06244401, -0.098911874, -0.017475998, -0.001042626, 0.07067651, 0.15693997, 0.04456361, 0.051365186, -0.008390114, -0.033872284, -0.019857459, -0.00072618574, 0....",0.651874
852,Dollars And Sense,explains why it’s so hard to manage money and teaches you how to combat false cues and natural desires so you can manage your dollars in better ways.,money,"[0.054979376, 0.0430027, -0.03762756, 0.07994847, -0.00085937895, -0.0025537603, 0.08726418, 0.023318874, 0.029843703, 0.067124225, -0.008823012, 0.017451871, -0.01763039, 0.0020111979, 0.05697426, -0.07714481, -0.006483417, -0.016270662, -0.06643053, 0.03177696, 0.045077104, -0.070066534, 0.012073329, 0.006504716, -0.0009769748, 0.067018084, -0.049844347, -0.040540904, -0.014434035, -0.03478365, 0.08638672, 0.081822775, 0.04643564, -0.04376575, -0.040282883, 0.0025635695, -0.04401676, 0.091...",0.641765
208,More Money Than God,"teaches us about the ins and outs of hedge funds, how those managing money makes a profit, and how you can learn from them and apply their techniques to your money management strategy.",biography,"[0.05313281, -0.045197617, -0.0405806, 0.04100588, 0.013576494, -0.021785595, 0.02766453, 0.031195011, -0.052767977, 0.055406563, -0.032817516, 0.07208235, -0.00563337, -0.01658589, 0.011596747, -0.017041737, -0.036353778, -0.0009390169, -0.02865233, -0.023240028, 0.04300684, -0.07096933, 0.037095822, -0.002504824, -0.06398871, -0.0032765544, 0.02379255, 0.012874897, -0.086575575, -0.024565164, 0.040140316, 0.002272003, -0.025026804, -0.043676045, -0.08525989, 0.06673008, 0.0016307747, 0.057...",0.578497
697,The Latte Factor,teaches us how to overcome limiting beliefs about money and build our financial freedom through small daily choices.,happiness,"[0.022473333, 0.10840659, -0.06586062, 0.06697642, 0.031095698, 0.039169848, 0.05713531, -0.022181872, 0.057183832, 0.034539305, 0.024854088, 0.09547495, -0.031554412, -0.024253825, 0.016520204, -0.059745155, -0.080963016, 0.047889713, -0.013357755, -0.0069780876, -0.004574534, -0.04366629, -0.005707922, 0.04708664, 0.0019194632, 0.039570026, 0.07239121, -0.05961553, 0.036727, -0.06526885, 0.060454894, 0.017766533, 0.029864775, -0.04063713, 0.022450551, 0.011211879, 0.074348964, 0.0458758, -...",0.564691


In [41]:
books_df.sample(10, random_state = 11)

,book_name,summaries,categories,embedding
591,Happier At Home,"is an instruction manual to transform your home into a castle of happiness by figuring out what needs to be changed, what needs to stay the same, and embracing the gift of family.",relationships,"[0.0077993437, 0.0932788, 0.004560748, 0.005866498, -0.055539086, 0.008825381, -0.03849979, -0.04745445, -0.05637232, -0.08027111, -0.011345213, 0.11436344, 0.008592182, -0.05887099, 0.013477055, -0.048856568, -0.07116568, -0.0059995134, 0.023942288, -0.0049593966, 0.022338722, 0.09703183, -0.057526603, 0.030811796, -0.023365648, 0.00060990686, 0.018511623, 0.01215557, 0.0027189248, -0.05335048, 0.032287363, 0.056207612, 0.057962567, -0.035237383, -0.047036637, 0.048395086, -0.012089409, 0.0..."
516,Finite And Infinite Games,"offers the theory that we play many different games in life, showing you that work and relationships are long-term endeavors and how to play them in order to win.",relationships,"[-0.058282398, 0.010690709, 0.035712983, -0.047647085, 0.012895821, 0.09223728, 0.060898226, -0.020687832, 0.060164206, -0.019814404, -0.10079765, 0.024563393, -0.022688052, 0.029424123, 0.06674728, -0.016588872, -0.003007681, -0.033017956, -0.022672053, -0.0540434, 0.0025823058, -0.08530395, -0.035254933, 0.00044264554, -0.088818446, 0.0158274, -0.024472872, 0.043738417, -0.07609076, 0.024542708, 0.0047117546, 0.06317997, 0.055127636, 0.0011014735, -0.03609698, 0.056625213, -0.055506546, 0...."
38,Brain Food,"delves into the topic of nutrition and how certain foods and nutrients can affect the well-being of the brain, its memory function, its cognitive capability, and how what we ingest can reverse the brain’s inclination to develop certain diseases.",science,"[0.039772194, 0.027661141, 0.019442856, 0.07824494, -0.045967255, 0.11908857, 0.08288415, 0.019878514, -0.07451506, -0.010377052, -0.059607424, 0.041752886, -0.05929903, -0.07998494, -0.0003792247, -0.0777932, 0.05592712, 0.047785647, -0.10217003, 0.019456934, 0.09984969, 0.010355108, 0.02339936, 0.019309305, -0.07046327, 0.043388054, 0.052078925, -0.08543924, -0.064577155, -0.04297099, 0.11343319, -0.012125223, 0.033574764, -0.002241329, -0.090921834, 0.0026565294, -0.035666827, -0.08094406..."
194,Sex at Dawn,"challenges conventional views on sex by diving deep into our ancestors’ sexual history and the rise of monogamy, thus prompting us to rethink our understanding of what sex and relationships should really feel and be like.",science,"[-0.067141995, 0.030640826, -0.013717521, 0.026724325, -0.067206524, 0.07483333, -0.033625267, 0.0071488353, 0.01226056, 0.032761455, 0.03898655, -0.011486932, -0.06533508, -0.0123630995, 0.05776935, -0.033949967, 0.0090097515, -0.031863585, -0.015984971, 0.10820672, 0.0066110045, -0.034166753, 0.09683709, -0.06772517, -0.047195956, 0.012363927, -0.031339347, -0.02427098, 0.001763585, 0.01784032, 0.008167862, 0.07720627, 0.036471907, 0.05197484, -0.08208541, 0.007448689, -0.048763692, -0.020..."
1142,The Long Tail,"explains why the big commercial hit is dead, how businesses can and will generate most of their future revenue from a long tail of niche products, which serve even the rarest customer needs, and what you can do to embrace this idea today.",marketing,"[-0.021558605, -0.07426372, 0.0009376021, -0.074445926, 0.067622416, 0.046283435, -0.0064062183, 0.08469913, -0.01977089, -0.07778842, -0.026505342, 0.061417256, 0.01566617, -0.0015896695, 0.08591657, -0.014861972, 0.0044025714, -0.024217408, -0.07089642, -0.013463614, -0.015105343, 0.018307883, -0.025101678, 0.00092167046, -0.1120635, 0.0060529476, -0.017921438, -0.05324288, -0.0073297946, -0.05583614, 0.027704107, 0.09744753, 0.029030716, -0.007086579, 0.02329727, -0.00015077283, -0.029898..."
566,The Longevity Project,"shows you how you can live longer by analyzing the results from one of the world’s longest-lasting studies and drawing surprising conclusions about the work ethic, happiness, love, marriage and religion of pe

In [42]:
get_similar_books( 'Permission Marketing' )

,book_name,summaries,categories,embedding,similarity
1142,The Long Tail,"explains why the big commercial hit is dead, how businesses can and will generate most of their future revenue from a long tail of niche products, which serve even the rarest customer needs, and what you can do to embrace this idea today.",marketing,"[-0.021558605, -0.07426372, 0.0009376021, -0.074445926, 0.067622416, 0.046283435, -0.0064062183, 0.08469913, -0.01977089, -0.07778842, -0.026505342, 0.061417256, 0.01566617, -0.0015896695, 0.08591657, -0.014861972, 0.0044025714, -0.024217408, -0.07089642, -0.013463614, -0.015105343, 0.018307883, -0.025101678, 0.00092167046, -0.1120635, 0.0060529476, -0.017921438, -0.05324288, -0.0073297946, -0.05583614, 0.027704107, 0.09744753, 0.029030716, -0.007086579, 0.02329727, -0.00015077283, -0.029898...",0.610961
931,Epic Content Marketing,shows why traditional methods for selling like TV and direct mail are dead and how creating content is the new future of advertising because it actually grabs people’s attention by focusing on what they care about instead of your product.,productivity,"[-0.01824778, -0.077334255, 0.019648952, -0.02451084, 0.08285821, 0.028583331, -0.050017953, 0.048232578, 0.0080745835, -0.047778886, -0.0047820327, 0.11049996, -0.019245721, 0.005509677, 0.051240288, -0.052823532, 0.039643344, -0.08599283, -0.031803805, 0.0037298973, 0.045756727, 0.036747202, 0.011806288, -0.0029037131, -0.017090403, -0.033031814, -0.04751452, -0.044055026, 0.03379205, -0.041031312, 0.010960794, 0.10099077, 0.033132248, -0.028805278, -0.06683315, 0.026692325, -0.0057818857,...",0.579648
1088,Purple Cow,"explains why building a great product and advertising the heck out of it simply doesn’t cut it anymore and how you can build something that’s so remarkable people have to share it, in order to succeed in today’s crowded post-advertising world.",psychology,"[-0.01760213, -0.0009883135, 0.02596374, -0.050047975, 0.12011608, 0.008536537, 0.05461069, 0.07618105, -0.019600889, -0.058138635, -0.0375484, 0.041674748, 0.020128738, -0.0082385605, 0.031226948, -0.0014112201, -0.0019882072, -0.06204705, -0.07186339, -0.08395643, 0.074250765, 0.0012193727, 0.0043972544, 0.048593376, -0.062215775, 0.039220553, -0.026271142, -0.012991842, 0.040553704, -0.057114813, 0.042564843, 0.019350361, 0.098678894, -0.02220773, 0.007858358, 0.03368367, -0.04420861, -0....",0.577623
1126,This Is Marketing,"argues that marketing success in today’s world comes from focusing more on the needs, values, and desires of our target audience, rather than spamming as many people as possible with our message.",marketing,"[0.021531021, -0.0069929776, -0.007194926, -0.021298204, 0.0699725, 0.0033442008, 0.12523745, 0.01318161, 0.02555298, -0.14000852, 0.039742045, 0.013115404, 0.07759699, -0.029995725, 0.1357686, -0.03337303, 0.05219284, -0.018406224, -0.07159449, 0.010958593, -0.06683711, -0.0070392443, -0.0008505618, 0.030236254, -0.12841374, -0.07746259, -0.042098552, -0.025428712, -0.008378049, -0.031613756, 0.037909944, 0.038204197, 0.06642594, 0.018680094, 0.053969078, 0.048373576, -0.070594005, -0.02528...",0.575825
1132,The Fortune Cookie Principle,"explains why a great product or service isn’t enough, how you can tell a compelling story about your brand and why that’s the most important aspect of running a business today.",marketing,"[0.022747673, 0.02911995, 0.044821016, -0.078607365, 0.05329563, 0.025894988, 0.039052155, 0.0967794, -0.0011926218, -0.054891624, -0.024087202, 0.09490095, 0.011908717, -0.0020536613, 0.052389715, -0.06703157, 0.07061497, -0.056515515, -0.09647855, -0.051603265, 0.08005686, -0.039464835, 0.024676263, 0.08539037, -0.11195084, 0.042205345, 0.014462398, 0.009545167, -0.038815986, -0.10019576, -0.043126456, 0.038978238, 0.065933816, -0.05899835, 0.028070543, 0.0014222779, 0.0010388482, -0.00243...",0.552454
